In [1]:
"""
BERTを用いたポジネガ分類器つくっちゃうよ

"""

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

from transformers import BertJapaneseTokenizer, BertModel, AutoModel
from transformers import TrainingArguments, Trainer

import pandas as pd
from sklearn.model_selection import train_test_split

MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
print(torch.cuda.is_available())

True


### 事前準備
学習データ成形、事前学習クラスのロード、BERTを使ったモデルのクラス作成

In [2]:
# ポジネガデータセット
df_dataset = pd.read_csv(
    'D:/DataSet/chABSA-dataset/chABSA-dataset/dataset.tsv',
    sep='\t', 
    header=None
).rename(columns={0:'text', 1:'label'}).loc[:, ['text', 'label']]

# ひとまずこういうFmtのデータに成形するところまでがんばる
df_dataset

,text,label
0,当社グループを取り巻く環境は、実質賃金が伸び悩むなか、消費者の皆様の生活防衛意識の高まりや節...,0
1,春から夏にかけましては個人消費の低迷などにより、きのこの価格は厳しい状況で推移いたしました,0
2,台湾の現地法人「台灣北斗生技股份有限公司」におきましては、ブランドの構築、企画提案などに力を...,0
3,化成品事業におきましては、引き続き厳しい販売環境にありましたが、中核である包装資材部門におき...,0
4,以上の結果、化成品事業の売上高は92億45百万円（同1.7％減）となりました,0
...,...,...
2808,当連結会計年度におきましては、連結子会社のデジタル・アドバタイジング・コンソーシアム株式会社...,1
2809,新規の自動ドアの売上台数は僅かに減少したものの、シートシャッターの大型物件に加え、取替の売上...,1
2810,"加えて、保守契約が堅調に増加し、売上高は6,952百万円（前年同期比1.2％増）となりました",1
2811,利益につきましては、取替工事の増加及び保守契約による安定的な利益の確保により、セグメント利益...,1


In [11]:
"""
BERTによる分類を行うレイヤのクラス

中に事前学習モデルを持ち、
input_ids -> model -> output -> classifier
という各レイヤのデータフローを流す
"""
class BertClassifier(nn.Module):
    def __init__(self, pretrained_model):
        super(BertClassifier, self).__init__()
        
        # 事前学習モデル
        self.model = pretrained_model
        
        # 線形変換層（全結合層）
        # ポジネガ（２カテゴリ）分類なので出力層は2
        self.classifier = nn.Linear(in_features=768, out_features=2)
        
        # 重み初期化
        nn.init.normal_(self.classifier.weight, std=0.02)
        nn.init.normal_(self.classifier.bias, 0)
    
#     def forward(self, input_ids):
    def forward(self, input_ids, labels, token_type_ids=None, attention_mask=None):
        output = self.model(input_ids)

        # トークン毎の特徴量（使わない）
        # last_hidden_state = output.last_hidden_state
        
        # 文代表（[CLS]トークン）の特徴量
        pooler_output = output.pooler_output
                
        # 分類タスク
        output_classifier = self.classifier(pooler_output)
        
        return output_classifier        
        

In [4]:
"""
https://dreamer-uma.com/pytorch-dataset/

対象タスクのデータを扱うDataset
データの格納と引き出し　DataLoaderと組み合わせてミニバッチ学習が可能
Datasetを自作する場合は必ず以下のメソッドを実装すること
__len__(): Datasetのサイズ（データ数）
__getitem__(): Datasetの要素にアクセス

"""
class PosiNegaDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = { k: torch.tensor(v[idx]) for k, v in self.encodings.items() }
#         item = { k: torch.tensor(v[idx]).cuda() for k, v in self.encodings.items() }
        item["labels"] = torch.tensor(self.labels[idx])
        return item

### 事前準備、以上
ここからは上で用意した各種クラスやモデルやデータセットを使ってタスクを解くコーディングをしていく

In [5]:
# トークナイザ
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 事前学習モデル
model = BertModel.from_pretrained(MODEL_NAME)
# model = model.cuda()  

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
sentence="Pytorchの仕組みを理解しながらBERTする"
encodings = tokenizer(sentence, add_special_tokens=True, return_tensors='pt')
encodings = { k: v.cuda() for k, v in encodings.items()} # テンソルなので辞書ループしてcudaる

encodings
# encodings['input_ids']
# tokenizer.convert_ids_to_tokens(encodings['input_ids'].tolist()[0])

{'input_ids': tensor([[    2,   318, 25034,   430,  1606,     5,  8957,    11,  3426,    15,
            895, 14773, 28680, 28642,    34,     3]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [6]:
# 特徴量X、ラベルyを取得
X, y = df_dataset["text"].values, df_dataset["label"].values

# train, val, test分割
# random_stateはシャッフルの乱数シード固定、stratifyは正例、負例のラベル数均一にする処理
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=0, stratify=y_val)

# トークナイザでモデルへのinputとなるようencodingする
max_len = 256 #512

enc_train = tokenizer(
    X_train.tolist(), 
    add_special_tokens=True, 
    max_length=max_len,
    padding='max_length',
    return_tensors='pt',
)
# tokenizer.convert_ids_to_tokens(enc_train['attention_masks'].tolist()[0])

enc_val = tokenizer(
    X_val.tolist(), 
    add_special_tokens=True, 
    max_length=max_len,
    padding='max_length',
    return_tensors='pt',
)

enc_test = tokenizer(
    X_test.tolist(), 
    add_special_tokens=True, 
    max_length=max_len,
    padding='max_length',
    return_tensors='pt',
)

In [7]:
# Datasetを作成
ds_train = PosiNegaDataset(enc_train, y_train)
ds_val = PosiNegaDataset(enc_val, y_val)
ds_test = PosiNegaDataset(enc_test, y_test)

# DataLoaderを作成
batch_size = 8
dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
dl_val = DataLoader(ds_val, batch_size=batch_size, shuffle=False)


In [13]:
ds_train.__getitem__(0)

C:\Users\licht\AppData\Local\Temp\ipykernel_14180\2971897890.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { k: torch.tensor(v[idx]) for k, v in self.encodings.items() }


{'input_ids': tensor([    2,    70,   124,    18,   455,     5,   854,     6,  6446,  9026,
             9,  4087, 28501,  1779,    25,   827,   127,  1625,   429,   758,
            23,  6089,   701,  2448,   143,   518,   648,     5,  2157,   222,
          1790,   455,     7, 16745,  5161,     9,  5076,  1779,    57,   827,
           429,   758,    23,  6089,   701,  4037,   143,   101,   648,     5,
          2157,   222, 12703,  9177,     7, 15310,    34,   162, 28692,  2947,
          5161,     9,  2538,  1779,   127,  1625,   429,   758,    23,  6089,
           701,  3374,   143,    17,   648,     5,  2157,    24,    13,   297,
          3913,    10,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [16]:
# 事前学習モデル
print(model.config)

# BERTモデル
bert_classifier = BertClassifier(model)

BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



In [13]:
# Trainerを作成
training_args = TrainingArguments(
    output_dir='./outputs',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    no_cuda=False,
)

trainer = Trainer(
    model=bert_classifier,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:
# ファインチューニング
trainer.train()

C:\Users\licht\anaconda3\envs\py39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2250
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 564
C:\Users\licht\AppData\Local\Temp\ipykernel_11824\2971897890.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { k: torch.tensor(v[idx]) for k, v in self.encodings.items() }


RuntimeError: grad can be implicitly created only for scalar outputs